In [1]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np

In [2]:
from szp_funcs import print_df, path_to_data, day_types, to_double

In [3]:
month = 'apr'
nday = 21

In [42]:
bad_stat = ['Отпуск по уходу за ребенком', 'Отпуск по беременности и родам', 'Увольнение']

In [5]:
res = pd.read_excel(path_to_data + month + '.xlsx')

In [6]:
res['stv'] = res.apply(lambda row: to_double(row, 'stv'), axis=1)

In [26]:
def give_norm(row):
    if row['status'] == 'Увольнение':
        return nday
    return row['nday']

In [27]:
res['nday'] = res.apply(lambda row: give_norm(row), axis=1)

In [18]:
print_df(res[(res.inn == 7736110982) & (res.type == day_types[2])],'qwe')

In [7]:
res['stv'].fillna(1, inplace=True)
res['nday'].fillna(1, inplace=True)
res['wday'].fillna(res['nday'], inplace=True)

In [35]:
res['spis'] = res.apply(lambda row: row['stv'] * row['wday'] / nday, axis=1)

In [41]:
res[(res.inn == 7736110982) & (res.status == 'Увольнение') & (res.type == day_types[2])]

,inn,snils,sum,job,day,type,stv,status,status_pref,wday,nday,spis
184018,7736110982,130-349-376 35,18721.46,Вожатый,1,Внешнее совместительство,0.5,Увольнение,NaN,10.0,6.0,0.238095
184135,7736110982,139-389-172 07,18721.46,Вожатый,1,Внешнее совместительство,0.5,Увольнение,NaN,10.0,6.0,0.238095
184136,7736110982,139-389-172 07,8948.88,Вожатый,1,Внешнее совместительство,0.5,Увольнение,NaN,5.0,5.0,0.119048
184139,7736110982,139-709-876 16,NaN,Вожатый,0,Внешнее совместительство,0.5,Увольнение,Увольнение,1.0,1.0,0.023810
184140,7736110982,139-709-876 16,23446.46,Вожатый,1,Внешнее совместительство,0.5,Увольнение,NaN,10.0,6.0,0.238095
184141,7736110982,139-709-876 16,7159.10,Вожатый,1,Внешнее совместительство,0.5,Увольнение,NaN,5.0,5.0,0.119048
184222,7736110982,146-583-805 93,7159.10,Вожатый,1,Внешнее совместительство,0.5,Увольнение,NaN,5.0,5.0,0.119048
184533,7736110982,177-400-211 46,7159.10,Вожатый,1,Внешнее совместительство,0.5,Увольнение,NaN,5.0,5.0,0.119048
184539,7736110982,177-577-052 17,22096.46,Вожатый,1,Внешнее совместительство,0.5,Увольнение,NaN,10.0,6.0,0.238095
184540,7736110982,177-577-052 17,8948.88,Вожатый,1,Внешнее совместительство,0.5,Увольнение,NaN,5.0,5.0,0.119048


In [47]:
ppl = res[res.status.isin(bad_stat) == False][['inn', 'type', 'snils']].groupby(['inn', 'type']).nunique().unstack().fillna(0.0)
ppl.index.names = ['inn']
ppl.columns = [s[1] + ' головы' for s in ppl.columns]

In [48]:
ppl[ppl.index == 7736110982]

,Внешнее совместительство головы,Внутреннее совместительство головы,Основное место работы головы
inn,,,
7736110982,78.0,282.0,1368.0


In [45]:
stv = res[res.status.isin(bad_stat) == False][['inn', 'type', 'stv']].groupby(['inn', 'type']).sum().unstack().fillna(0.0)
stv.index.names = ['inn']
stv.columns = [s[1] + ' ставки' for s in stv.columns]


In [46]:
stv[stv.index == 7736110982]

,Внешнее совместительство ставки,Внутреннее совместительство ставки,Основное место работы ставки
inn,,,
7736110982,35.633333,100.017233,1415.097


In [36]:
spis = res[res.status.isin(bad_stat) == False][['inn', 'type', 'spis']].groupby(['inn', 'type']).sum().unstack().fillna(0.0)
spis.index.names = ['inn']
spis.columns = [s[1] + ' списоч' for s in spis.columns]

In [37]:
spis[spis.index == 7736110982]

,Внешнее совместительство списоч,Внутреннее совместительство списоч,Основное место работы списоч
inn,,,
7736110982,33.760794,100.415186,1347.583095


In [43]:
print_df(pd.merge(pd.merge(ppl, stv, on='inn'), spis, on='inn'), 'Сравнение количества людей ' + month)